In [14]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Circle
import os

In [2]:
def ruku_schritt(f, y, h):
    k1 = f(y)
    k2 = f(y + 0.5 * h * k1)
    k3 = f(y + 0.5 * h * k2)
    k4 = f(y + h * k3)
    return y + (h / 6) * (k1 + 2 * k2 + 2 * k3 + k4)

In [ ]:
# Die Eingangsgröße für y sollte folgende Form haben: 
# y = [x1_x, x1_y, x2_x, x2_y, v1_x, v1_y, v2_x, v2_y]
# m1 und m2 sind natürlich skalar
def rechte_seite(y, m1, m2):
    x1 = y[0:2]
    x2 = y[2:4]
    v1 = y[4:6]
    v2 = y[6:8]

    r = x2 - x1
    r_norm = np.linalg.norm(r)
    #if r_norm == 0:
    #    F1 = np.zeros(2)
    #else:

    F1 = (m1 * m2 / r_norm**3) * r
    F2 = -F1

    return np.concatenate([v1, v2, F1 / m1, F2 / m2])

In [20]:
def erzeuge_frame(x_1, x_2, r1, r2, name):
    fig, ax = plt.subplots()
    ax.add_patch(Circle(x_1, r1, fill=True, color='blue'))
    ax.add_patch(Circle(x_2, r2, fill=True, color='red'))
    ax.set_aspect('equal')
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    ax.axis('off') 

    os.makedirs(os.path.dirname(name), exist_ok=True)

    plt.savefig(name, dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
# teste erzeuge frame
name = "testframe/test.jpg"
x_1 = (2,0)
x_2 = (0, 2)
erzeuge_frame(x_1, x_2, 0.5, 0.1, name)

In [28]:
pip install imageio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
import imageio.v2 as imageio

def erstelle_gif(frame_ordner, ausgabepfad, delay_ms=100):
    bilder = []
    dateien = sorted(os.listdir(frame_ordner))
    for datei in dateien:
        pfad = os.path.join(frame_ordner, datei)
        bilder.append(imageio.imread(pfad))
    imageio.mimsave(ausgabepfad, bilder, duration=delay_ms / 1000)

In [34]:
def doppelstern(m1, m2, x1_init, x2_init, p, h):
    y = np.array([
        x1_init, 0.0,  # x1
        x2_init, 0.0,  # x2
        0.0, p / m1,   # v1
        0.0, -p / m2   # v2
    ])
    
    f = lambda y: rechte_seite(y, m1, m2)
    for i in range(1000):
        name = f"frames/frame_{i+1}"
        x1 = y[0:2]
        x2 = y[2:4]
        r1 = m1**(2/3)*0.05 # Hier ist es der Planetenradius
        r2 = m2**(2/3)*0.05
        erzeuge_frame(x1, x2, r1, r2, name)
        y = ruku_schritt(f, y, h)

    erstelle_gif("frames", "doppelstern.gif", 10)


In [35]:
doppelstern(1, 5, -1, 1, 1, 0.01)